### Vaishnavi Sanjana Karri

In [1]:
import nltk
nltk.download('stopwords')
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to C:\Users\Rupesh
[nltk_data]     ChV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Rupesh ChV\anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [2]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [3]:
# Read in the Pandas dataframe

df = pd.read_csv("C:\\Users\\Rupesh ChV\\OneDrive\\Desktop\\CSV_path.csv")
print(df.columns)
display(df)

Index(['University', 'Course Title', 'Link', 'Description',
       'General Course Catalogue'],
      dtype='object')


,University,Course Title,Link,Description,General Course Catalogue
0,Arizona State University,Principles of Public Interest Technology,NaN,NaN,NaN
1,Boston University,XCC433: Justice Media co-Lab,https://www.bu.edu/cds-faculty/programs/justic...,The Justice Media co-Lab provides the programm...,https://www.bu.edu/phpbin/course-search/
2,Boston University,Law & Algorithms,https://www.bu.edu/law/courses/lawjd673/,This cross-cutting and interdisciplinary cours...,NaN
3,Boston University,Data Science for Politics,https://www.bu.edu/academics/cas/courses/cas-p...,Data science is changing how we understand and...,NaN
4,Cal Poly State,Technically Human: The Art and the Ethics of T...,https://debdonig.wordpress.com/technically-hum...,"In this course, we will look critically at the...",NaN
...,...,...,...,...,...
110,University of Washington,Value Sensitive Design,NaN,Undergraduate course,NaN
111,NYU,Data Science for Everyone,https://cds.nyu.edu/ds-for-everyone/,Data Science for Everyone is the flagship unde...,NaN
112,U of M,SEE LIST OF ALL UNDERGRAD COURSES:,https://stpp.fordschool.umich.edu/undergraduat...,NaN,NaN
113,Stanford,SOC 141P: Public Interest Tech: Case Studies (...,https://explorecourses.stanford.edu/search?aca...,What does public interest technology look like...,NaN


In [5]:
# NLTK Stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["also", "and", "its", "bring", "got", "from", "lot", "rather", "even", "from", "but","that","in","you","who","to", "of", "become", "thats", "looking", "Nan"])

In [6]:
# Convert to list 
data = df['Description'].values.tolist()

In [7]:
# Tokenize words and cleanup the text

def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[1])

['the', 'justice', 'media', 'co', 'lab', 'provides', 'the', 'programmatic', 'structure', 'to', 'train', 'new', 'generation', 'of', 'computational', 'investigative', 'journalists', 'equipped', 'to', 'leverage', 'the', 'power', 'of', 'computing', 'and', 'data', 'science', 'to', 'advance', 'justice', 'and', 'transparency', 'the', 'first', 'curricular', 'collaboration', 'between', 'cds', 'and', 'other', 'disciplines', 'this', 'co', 'lab', 'aims', 'to', 'develop', 'and', 'test', 'an', 'institutional', 'framework', 'for', 'collaborations', 'faculty', 'students', 'and', 'external', 'partners', 'around', 'courses', 'and', 'summer', 'programs', 'that', 'lead', 'to', 'student', 'placements', 'in', 'pit', 'careers']


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

['the', 'justice', 'media', 'co', 'lab', 'provides', 'the', 'programmatic', 'structure', 'to', 'train', 'new', 'generation', 'of', 'computational', 'investigative', 'journalists', 'equipped', 'to', 'leverage', 'the', 'power', 'of', 'computing', 'and', 'data', 'science', 'to', 'advance', 'justice', 'and', 'transparency', 'the', 'first', 'curricular', 'collaboration', 'between', 'cds', 'and', 'other', 'disciplines', 'this', 'co', 'lab', 'aims', 'to', 'develop', 'and', 'test', 'an', 'institutional', 'framework', 'for', 'collaborations', 'faculty', 'students', 'and', 'external', 'partners', 'around', 'courses', 'and', 'summer', 'programs', 'that', 'lead', 'to', 'student', 'placements', 'in', 'pit', 'careers']


In [9]:
# Define function for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Form trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1])

['justice', 'medium', 'co', 'lab', 'provide', 'programmatic', 'structure', 'train', 'new', 'generation', 'computational', 'investigative', 'journalist', 'equip', 'leverage', 'power', 'computing', 'data', 'science', 'advance', 'justice', 'transparency', 'first', 'curricular', 'collaboration', 'cds', 'discipline', 'co', 'lab', 'aim', 'develop', 'test', 'institutional', 'framework', 'collaboration', 'faculty', 'student', 'external', 'partner', 'course', 'summer', 'program', 'lead', 'student', 'placement', 'pit', 'career']


#### Creating LDA Models

##### 1. Using lemmatized corpus

In [11]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  

# Create Corpus 
texts = data_lemmatized  

# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  

# View 
print(corpus[1])

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1)]


In [12]:
id2word[24]

'lab'

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Print the keyword of topics

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"course" + 0.017*"computer" + 0.016*"teach" + 0.015*"science" + '
  '0.010*"make" + 0.010*"social" + 0.009*"school" + 0.009*"develop" + '
  '0.008*"sociology" + 0.008*"scientist"'),
 (1,
  '0.027*"technology" + 0.017*"course" + 0.017*"public" + 0.014*"student" + '
  '0.012*"work" + 0.011*"information" + 0.010*"system" + 0.009*"datum" + '
  '0.009*"policy" + 0.009*"design"'),
 (2,
  '0.035*"project" + 0.026*"student" + 0.024*"team" + 0.023*"partner" + '
  '0.021*"organization" + 0.020*"course" + 0.017*"base" + 0.016*"staff" + '
  '0.012*"policy" + 0.011*"science"'),
 (3,
  '0.054*"course" + 0.038*"nan" + 0.024*"undergraduate" + 0.023*"science" + '
  '0.022*"undergrad" + 0.017*"develop" + 0.016*"long" + 0.016*"data" + '
  '0.011*"semester" + 0.009*"summer"'),
 (4,
  '0.034*"datum" + 0.023*"science" + 0.018*"course" + 0.017*"class" + '
  '0.014*"include" + 0.013*"decision" + 0.013*"student" + 0.010*"learn" + '
  '0.010*"privacy" + 0.009*"context"')]


In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.201368624412572

Coherence Score:  0.5700174478766286


In [16]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

C:\Users\Rupesh ChV\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.029220 -0.128225       1        1  60.496958
4     -0.059244 -0.085393       2        1  20.001549
0     -0.050046  0.055838       3        1   8.320522
2      0.168013  0.060002       4        1   7.070261
3     -0.087943  0.097777       5        1   4.110711, topic_info=        Term       Freq      Total Category  logprob  loglift
9     course  89.000000  89.000000  Default  30.0000  30.0000
36   science  41.000000  41.000000  Default  29.0000  29.0000
61     datum  57.000000  57.000000  Default  28.0000  28.0000
38   student  58.000000  58.000000  Default  27.0000  27.0000
272  project  16.000000  16.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
9     course   9.789460  89.266035   Topic5  -2.9110   0.9813
36   science   4.186800  41.739541   Topic5  -3.7604   0.8921
61     datum   1.396286  57.566533   Topic5  -4.8585  -0.5276
286     take   1.390959   7.336829   Topic5  -4.8623   1.5287
588    major   1.390959   9.453299   Topic5  -4.8623   1.2752

[267 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1389      2  0.936824     active
387       1  0.966548    address
2         1  0.572829        aim
2         4  0.381886        aim
44        2  0.830171  algorithm
...     ...       ...        ...
225       4  0.059265       work
173       1  0.676895      world
173       2  0.169224      world
173       3  0.112816      world
401       5  0.721985       year

[317 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 3, 4])

##### 2. Using Bigram corpus

In [17]:
# Create Dictionary 
id2word_bi = corpora.Dictionary(data_words_bigrams)  

# Create Corpus 
texts_bi = data_words_bigrams  

# Term Document Frequency 
corpus_bi = [id2word.doc2bow(text) for text in texts_bi]  

# View 
print(corpus_bi[1])

[(1, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (10, 1), (11, 1), (12, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (23, 2), (24, 2), (25, 1), (26, 1), (28, 1), (30, 1), (32, 1), (34, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]


In [18]:
id2word_bi[24]

'journalists'

In [19]:
lda_model_bi = gensim.models.ldamodel.LdaModel(corpus=corpus_bi,
                                           id2word=id2word_bi,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the keyword of topics

pprint(lda_model_bi.print_topics())
doc_lda_bi = lda_model_bi[corpus_bi]

[(0,
  '0.010*"computational" + 0.008*"identity" + 0.005*"implications" + '
  '0.005*"shape" + 0.005*"changing" + 0.004*"reasoning" + 0.004*"techniques" + '
  '0.004*"leverage" + 0.003*"hopes" + 0.003*"digester"'),
 (1,
  '0.059*"computational" + 0.030*"disruptions" + 0.013*"advertising" + '
  '0.008*"programs" + 0.008*"answer" + 0.008*"structure" + 0.008*"economic" + '
  '0.007*"external" + 0.007*"iteration" + 0.007*"angles"'),
 (2,
  '0.033*"computational" + 0.014*"intelligence" + 0.009*"sectoral" + '
  '0.009*"regimes" + 0.008*"solutions" + 0.007*"programs" + '
  '0.007*"complexities" + 0.007*"commercial" + 0.006*"leverage" + '
  '0.006*"transparent"'),
 (3,
  '0.057*"courses" + 0.031*"programs" + 0.021*"computational" + '
  '0.014*"solutions" + 0.013*"expert" + 0.011*"legal" + 0.008*"health" + '
  '0.008*"complexities" + 0.008*"industrial" + 0.008*"industry"'),
 (4,
  '0.031*"machine" + 0.025*"entities" + 0.022*"sectoral" + 0.022*"nan" + '
  '0.019*"computational" + 0.018*"industri

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_bi.log_perplexity(corpus_bi))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda_bi = CoherenceModel(model=lda_model_bi, texts=data_words_bigrams, dictionary=id2word_bi, coherence='c_v')
coherence_lda_bi = coherence_model_lda_bi.get_coherence()
print('\nCoherence Score: ', coherence_lda_bi)


Perplexity:  -7.224387913538946

Coherence Score:  0.698700849262069


In [22]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis_bi = gensimvis.prepare(lda_model_bi, corpus_bi, id2word_bi)
vis_bi

C:\Users\Rupesh ChV\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.040076 -0.127371       1        1  39.197487
2      0.046247  0.019168       2        1  25.058958
4     -0.170443  0.002755       3        1  13.917137
1      0.048563  0.072147       4        1  10.932968
0      0.035556  0.033301       5        1  10.893450, topic_info=              Term       Freq      Total Category  logprob  loglift
11         courses  74.000000  74.000000  Default  30.0000  30.0000
9    computational  82.000000  82.000000  Default  29.0000  29.0000
90         machine  19.000000  19.000000  Default  28.0000  28.0000
400    disruptions  11.000000  11.000000  Default  27.0000  27.0000
272       entities  11.000000  11.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
255         beyond   0.850916  10.941994   Topic5  -5.9726  -0.3370
113          study   0.791902   3.894532   Topic5  -6.0444   0.6241
139         policy   0.791902   2.782459   Topic5  -6.0444   0.9604
152            art   0.791902   2.714675   Topic5  -6.0444   0.9850
159       consider   0.791902   4.450878   Topic5  -6.0444   0.4906

[282 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
46        1  0.186858     access
46        3  0.747432     access
345       1  0.375267   activity
345       2  0.375267   activity
345       5  0.187633   activity
...     ...       ...        ...
671       5  0.524585    whether
250       1  0.177768       work
250       3  0.533305       work
250       4  0.177768       work
1382      4  0.703376  workplace

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])

##### 3. Using Trigram corpus

In [23]:
# Create Dictionary 
id2word_tri = corpora.Dictionary(data_words_trigrams)  

# Create Corpus 
texts_tri = data_words_trigrams  

# Term Document Frequency 
corpus_tri = [id2word.doc2bow(text) for text in texts_tri]  

# View 
print(corpus_tri[1])

[(1, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (10, 1), (11, 1), (12, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (23, 2), (24, 2), (25, 1), (26, 1), (28, 1), (30, 1), (32, 1), (34, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]


In [24]:
id2word_tri[24]

'journalists'

In [25]:
lda_model_tri = gensim.models.ldamodel.LdaModel(corpus=corpus_tri,
                                                id2word=id2word_tri,
                                                num_topics=5,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=50,
                                                alpha='auto',
                                                per_word_topics=True)

In [26]:
# Print the keyword of topics

pprint(lda_model_tri.print_topics())
doc_lda_tri = lda_model_tri[corpus_tri]

[(0,
  '0.010*"computational" + 0.008*"identity" + 0.005*"implications" + '
  '0.005*"shape" + 0.005*"changing" + 0.004*"reasoning" + 0.004*"techniques" + '
  '0.004*"leverage" + 0.003*"hopes" + 0.003*"digester"'),
 (1,
  '0.059*"computational" + 0.030*"disruptions" + 0.013*"advertising" + '
  '0.008*"programs" + 0.008*"answer" + 0.008*"structure" + 0.008*"economic" + '
  '0.007*"external" + 0.007*"iteration" + 0.007*"angles"'),
 (2,
  '0.033*"computational" + 0.014*"intelligence" + 0.009*"sectoral" + '
  '0.009*"regimes" + 0.008*"solutions" + 0.007*"programs" + '
  '0.007*"complexities" + 0.007*"commercial" + 0.006*"leverage" + '
  '0.006*"transparent"'),
 (3,
  '0.057*"courses" + 0.031*"programs" + 0.021*"computational" + '
  '0.014*"solutions" + 0.013*"expert" + 0.011*"legal" + 0.008*"health" + '
  '0.008*"complexities" + 0.008*"industrial" + 0.008*"industry"'),
 (4,
  '0.031*"machine" + 0.025*"entities" + 0.022*"sectoral" + 0.022*"nan" + '
  '0.019*"computational" + 0.018*"industri

In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tri.log_perplexity(corpus_tri))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda_tri = CoherenceModel(model=lda_model_tri, texts=data_words_trigrams, dictionary=id2word_tri, coherence='c_v')
coherence_lda_tri = coherence_model_lda_tri.get_coherence()
print('\nCoherence Score: ', coherence_lda_tri)


Perplexity:  -7.224387913538946

Coherence Score:  0.698700849262069


In [28]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis_tri = gensimvis.prepare(lda_model_tri, corpus_tri, id2word_tri)
vis_tri

C:\Users\Rupesh ChV\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.040076 -0.127371       1        1  39.197487
2      0.046247  0.019168       2        1  25.058958
4     -0.170443  0.002755       3        1  13.917137
1      0.048563  0.072147       4        1  10.932968
0      0.035556  0.033301       5        1  10.893450, topic_info=              Term       Freq      Total Category  logprob  loglift
11         courses  74.000000  74.000000  Default  30.0000  30.0000
9    computational  82.000000  82.000000  Default  29.0000  29.0000
90         machine  19.000000  19.000000  Default  28.0000  28.0000
400    disruptions  11.000000  11.000000  Default  27.0000  27.0000
272       entities  11.000000  11.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
255         beyond   0.850916  10.941994   Topic5  -5.9726  -0.3370
113          study   0.791902   3.894532   Topic5  -6.0444   0.6241
139         policy   0.791902   2.782459   Topic5  -6.0444   0.9604
152            art   0.791902   2.714675   Topic5  -6.0444   0.9850
159       consider   0.791902   4.450878   Topic5  -6.0444   0.4906

[282 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
46        1  0.186858     access
46        3  0.747432     access
345       1  0.375267   activity
345       2  0.375267   activity
345       5  0.187633   activity
...     ...       ...        ...
671       5  0.524585    whether
250       1  0.177768       work
250       3  0.533305       work
250       4  0.177768       work
1382      4  0.703376  workplace

[344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])